In [3]:
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
import time, math, random
from typing import Tuple

import gym

In [4]:
env = gym.make("CartPole-v1")

In [4]:
policy = lambda x: 1
for _ in range(3):
    obs = env.reset()
    for _ in range(80):
        action = policy(obs)
        obs, reward, done, info = env.step(action)
        env.render()
        time.sleep(0.05)
#env.close()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pygame/surfarray.py:53: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if hasattr(numpy, type_name):
2022-06-16 00:21:58.514 Python[5098:6729787] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/w6/0d0dzq4d3t9g78b6qc727s4w0000gn/T/org.python.python.savedState
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gym/envs/classic_control/cartpole.py:168: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receiv

In [8]:
policy = lambda obs : int(obs[3] > 0)

In [14]:
n_bins = (6,4)

lower_bounds = [env.observation_space.low[2], -math.radians(50)]
upper_bounds = [env.observation_space.high[2], math.radians(50)]

def discretizer( _, __, angle, pole_velocity) -> Tuple[int, ...]:
    est = KBinsDiscretizer(n_bins = n_bins, encode = 'ordinal', strategy = 'uniform')
    est.fit([lower_bounds,upper_bounds])
    return tuple(map(int,est.transform([[angle,pole_velocity]])[0]))

In [6]:
Q_table = np.zeros(n_bins + (env.action_space.n,))
Q_table.shape

(6, 12, 2)

In [9]:
def policy(state : tuple):
    return np.argmax(Q_table[state])

In [10]:
def new_Q_value( reward: float, new_state : tuple, discount_factor=1) -> float:
    future_optimal_value = np.max(Q_table[new_state])
    learned_value = reward + discount_factor * future_optimal_value
    return learned_value

In [11]:
def learning_rate(n : int, min_rate=0.01) -> float:
    return max(min_rate, min(1.0,1.0-math.log10((n+1)/25)))

In [12]:
def exploration_rate(n: int, min_rate = 0.1) -> float:
    return max(min_rate, min(1, 1.0 - math.log10(n+1)/25))

In [16]:
n_episodes = 1000
scores = []
for e in range(n_episodes):
    current_state, done = discretizer(*env.reset()), False

    total_reward = 0

    while not done:
        action = policy(current_state)

        if np.random.random() < exploration_rate(e):
            action = env.action_space.sample()
        
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        if(done):
            reward = -100
        new_state = discretizer(*obs)

        lr = learning_rate(e)
        learnt_value = new_Q_value(reward,new_state)
        old_value = Q_table[current_state][action]
        Q_table[current_state][action] = (1-lr)*old_value + lr * learnt_value
    
    scores += [total_reward]

    if e % 1000 == 0:
        print(f"Episode {e}, rewards {scores[e-10:e-1]}")

Episode 0, rewards []


In [17]:
total_reward = 0

current_state, done = discretizer(*env.reset()), False
while not done:
    action = policy(current_state)

    # if np.random.random() < exploration_rate(e):
    #     action = env.action_space.sample()
    
    obs, reward, done, _ = env.step(action)
    total_reward += reward
    # new_state = discretizer(*obs)
    env.render()

    # lr = learning_rate(e)
    # learnt_value = new_Q_value(reward,new_state)
    # old_value = Q_table[current_state][action]
    # Q_table[current_state][action] = (1-lr)*old_value + lr * learnt_value
print(total_reward)

2022-06-16 02:57:42.287 Python[7670:6783978] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/w6/0d0dzq4d3t9g78b6qc727s4w0000gn/T/org.python.python.savedState


10.0


: 